In [1]:
import keras
import tensorflow as tf
from keras import models
from keras import layers
from keras import Input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
traindir = 'train'
validationdir = 'validation'

datagen = ImageDataGenerator(rescale=1/255)
batch_size=1
train_iter = datagen.flow_from_directory(traindir, class_mode='categorical', batch_size=batch_size)
val_iter = datagen.flow_from_directory(validationdir, class_mode='categorical', batch_size=batch_size)

train_total=train_iter.n

Found 70295 images belonging to 38 classes.
Found 17572 images belonging to 38 classes.


In this model, I wanted to make an ensemble of convolutional models. Initially, I had four branches, inspired by inception modules. When I realized that this was too complex of a network, my first hope was that reducing the batch size would allow the network to have an easier time computing. I then reduced the number of branches from four to three. Finally, with just two convolutional branches and a batch size of 1, I realized that my computer does not have enough computing power to crunch an ensemble model like this one, no matter how simplified I make it. Thus I have left this model in the file - it is not a failed model, just do not have the resources to properly run it. I think with enough tweaking this model can absolutely do better than the standard convolutional model can. 

In [3]:
data = Input(shape=(256, 256, 3))

branch_a = layers.Conv2D(64, (1,1), activation='relu', padding='same')(data)
branch_a = layers.Conv2D(64, (3,3), activation='relu', padding='same')(branch_a)

branch_b = layers.MaxPooling2D((3,3), strides=(1,1), padding='same')(data)
branch_b = layers.Conv2D(64, (1,1), activation='relu', padding='same')(branch_c)

output = layers.concatenate([branch_a, branch_b], axis=2)
output = layers.Flatten()(output)
out = layers.Dense(38, activation='softmax')(output)

model = models.Model(inputs=data, outputs=out)
model.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Below is shown one of the errors encountered; the usual error, however, was ResourceExhaustedError. They all indicate the same thing - that the model is too large to fit in local memory. 

In [4]:
model.fit_generator(train_iter, steps_per_epoch=int(train_total/batch_size), epochs=5, 
                    validation_steps=5, validation_data=val_iter)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/5


InternalError: 2 root error(s) found.
  (0) Internal: cudnn PoolForward launch failed
	 [[{{node max_pooling2d_1/MaxPool}}]]
	 [[metrics/acc/Mean/_79]]
  (1) Internal: cudnn PoolForward launch failed
	 [[{{node max_pooling2d_1/MaxPool}}]]
0 successful operations.
0 derived errors ignored.